In [94]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFilter
import json
import csv

Annotating a Sample Image

In [95]:
epidural_csv_file = 'segmentations/Results_Epidural Hemorrhage Detection_2020-11-16_21.31.26.148.csv'
epidural_image_repo = 'epidural/max_contrast_window'
epidural_image_file = 'ID_004c4b319.jpg'

In [96]:
def get_coords(image, image_csv_file, image_file_name):
    labels = get_labels_csv(image_csv_file, image_file_name)
    coord_list = []
    for label in labels:
        coord = (label['x'] * image.size[0], label['y'] * image.size[1])
        coord_list.append(coord)
    return coord_list
def get_labels_csv(image_csv_file, image_file_name):
    with open(image_csv_file) as csv_file:
        data_reader = csv.reader(csv_file, dialect='excel')
        for row in data_reader:
            if row[1] == image_file_name:
                return json.loads(row[7])
    return None

In [97]:
im = Image.open(f'{epidural_image_repo}/{epidural_image_file}')
features = []
for x in range(im.size[0]):
    for y in range(20):
        features.append(im.getpixel((x, y))[0])
        features.append(im.getpixel((x, y))[1])
        features.append(im.getpixel((x, y))[2])
draw = ImageDraw.Draw(im)
draw.line(get_coords(im, epidural_csv_file, epidural_image_file), fill='red')
im.show()

In [98]:
# the line of code below loads the 'all labels' field in the csv file in json
# coords_lists = json.loads(row[4].replace("'[", "[").replace("]'", "],").replace(',]', ']'))

Reading an Image and Its Classification

In [99]:
num_features = 786432

In [100]:
all_labels = pd.read_csv('segmentations/hemorrhage-labels.csv')

In [101]:
all_labels

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000012eaf,0,0,0,0,0,0
1,ID_000039fa0,0,0,0,0,0,0
2,ID_00005679d,0,0,0,0,0,0
3,ID_00008ce3c,0,0,0,0,0,0
4,ID_0000950d7,0,0,0,0,0,0
...,...,...,...,...,...,...,...
752798,ID_ffff82e46,0,0,0,0,0,0
752799,ID_ffff922b9,1,0,0,1,0,0
752800,ID_ffffb670a,1,0,0,0,1,0
752801,ID_ffffcbff8,0,0,0,0,0,0


In [102]:
labels = all_labels[all_labels['any'] == 1]

In [103]:
labels

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
14,ID_0002081b6,1,0,1,0,0,0
24,ID_0002a38ad,1,0,0,0,1,1
33,ID_000346ce2,1,0,0,0,0,1
36,ID_00042829c,1,0,0,1,0,0
43,ID_0004a5701,1,0,0,0,0,1
...,...,...,...,...,...,...,...
752755,ID_fffc60817,1,0,1,1,0,0
752769,ID_fffd00949,1,0,0,0,1,0
752783,ID_fffe2edb8,1,0,1,1,0,0
752799,ID_ffff922b9,1,0,0,1,0,0


In [104]:
epidural = labels[labels['epidural'] == 1 && labels['intraparenchymal'] == 1 && labels[]

In [105]:
epidural

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
164,ID_000edbf38,1,1,0,0,0,0
271,ID_00178eb80,1,1,0,0,0,0
320,ID_001bb2c00,1,1,0,0,0,0
443,ID_0026de01c,1,1,0,0,0,0
828,ID_004966d37,1,1,1,1,0,0
...,...,...,...,...,...,...,...
752300,ID_ffd27f702,1,1,0,0,0,0
752319,ID_ffd4cf367,1,1,0,0,1,0
752334,ID_ffd67050a,1,1,0,0,0,0
752455,ID_ffe17f20d,1,1,0,0,0,0


In [89]:
epidural['Image']

164       ID_000edbf38
271       ID_00178eb80
320       ID_001bb2c00
443       ID_0026de01c
828       ID_004966d37
              ...     
752300    ID_ffd27f702
752319    ID_ffd4cf367
752334    ID_ffd67050a
752455    ID_ffe17f20d
752721    ID_fff9bb57a
Name: Image, Length: 3145, dtype: object

In [106]:
epidural.shape

(3145, 7)

In [64]:
epidural_data = np.zeros((epidural.shape[0], num_features))

In [65]:
epidural_data.shape

(3145, 786432)

In [116]:
for index, value in enumerate(epidural['Image'].items()):
    print(f'epidural/max_contrast_window/{value[1]}.jpg')
    epidural_data[index, :] = plt.imread(f'epidural/max_contrast_window/{value[1]}.jpg').reshape(1,-1)
#for i in range(epidural_data.shape[0]):
#    epidural_data[i, :] = plt.imread('epidural/max_contrast_window/ID_000edbf38.jpg').reshape(1,-1)

epidural/max_contrast_window/ID_000edbf38.jpg
epidural/max_contrast_window/ID_00178eb80.jpg
epidural/max_contrast_window/ID_001bb2c00.jpg
epidural/max_contrast_window/ID_0026de01c.jpg
epidural/max_contrast_window/ID_004966d37.jpg


FileNotFoundError: [Errno 2] No such file or directory: 'epidural/max_contrast_window/ID_004966d37.jpg'

In [109]:
for row in np.nditer(epidural):
    print(row)
    break

TypeError: Iterator operand or requested dtype holds references, but the REFS_OK flag was not enabled